In [1]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

In [4]:
%%sql
drop table if exists SHAIN_SHUTOKU_SHIKAKU;
CREATE TABLE SHAIN_SHUTOKU_SHIKAKU (
    社員コード CHAR(4),
    資格     CHAR(2)
);
INSERT INTO SHAIN_SHUTOKU_SHIKAKU (社員コード, 資格) VALUES ('S001', 'FE');
INSERT INTO SHAIN_SHUTOKU_SHIKAKU (社員コード, 資格) VALUES ('S001', 'AP');
INSERT INTO SHAIN_SHUTOKU_SHIKAKU (社員コード, 資格) VALUES ('S001', 'DB');
INSERT INTO SHAIN_SHUTOKU_SHIKAKU (社員コード, 資格) VALUES ('S002', 'FE');
INSERT INTO SHAIN_SHUTOKU_SHIKAKU (社員コード, 資格) VALUES ('S002', 'SM');
INSERT INTO SHAIN_SHUTOKU_SHIKAKU (社員コード, 資格) VALUES ('S003', 'FE');
INSERT INTO SHAIN_SHUTOKU_SHIKAKU (社員コード, 資格) VALUES ('S004', 'AP');
INSERT INTO SHAIN_SHUTOKU_SHIKAKU (社員コード, 資格) VALUES ('S005', NULL); -- NULL値は引用符で囲みません

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [6]:
%%sql
SELECT * FROM SHAIN_SHUTOKU_SHIKAKU;

*  postgresql://padawan:***@db:5432/dsdojo_db
8 rows affected.


社員コード,資格
S001,FE
S001,AP
S001,DB
S002,FE
S002,SM
S003,FE
S004,AP
S005,None


### Q: 以下の、FEとAPを資格として持つ社員の結果を作るために必要なクエリは？


| 社員コード | 資格1 | 資格2 |
| :---: | :---: | :---: |
| S001 | FE | AP |
| S002 | FE | NULL |
| S003 | FE | NULL |


In [21]:
%%sql
-- OUTER JOINを使って資格の組み合わせ表を作る
SELECT 
    T1.社員コード,
    T1.資格 AS 資格1,
    T2.資格 AS 資格2
FROM 
    SHAIN_SHUTOKU_SHIKAKU AS T1
LEFT OUTER JOIN SHAIN_SHUTOKU_SHIKAKU AS T2 
    ON T1.社員コード = T2.社員コード;

*  postgresql://padawan:***@db:5432/dsdojo_db
16 rows affected.


社員コード,資格1,資格2
S001,FE,FE
S001,FE,AP
S001,FE,DB
S001,AP,FE
S001,AP,AP
S001,AP,DB
S001,DB,FE
S001,DB,AP
S001,DB,DB
S002,FE,FE


In [18]:
%%sql
-- A1 WHERE句でFEをもつ社員をフィルタした後, ON句でFEかつAPの組み合わせを抽出
SELECT 
    T1.社員コード,
    T1.資格 AS 資格1,
    T2.資格 AS 資格2
FROM 
    SHAIN_SHUTOKU_SHIKAKU AS T1
LEFT OUTER JOIN SHAIN_SHUTOKU_SHIKAKU AS T2 
    ON T1.社員コード = T2.社員コード 
        AND T1.資格 = 'FE' 
        AND T2.資格 = 'AP'
WHERE
    T1.資格 = 'FE';

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


社員コード,資格1,資格2
S001,FE,AP
S002,FE,None
S003,FE,None


In [17]:
%%sql
-- A2 FEをもつ社員をフィルタした後, APの資格を持つかどうかで資格2を設定
SELECT 
    T1.社員コード,
    T1.資格 AS 資格1,
    CASE 
        WHEN T2.資格 = 'AP' THEN 'AP'
        ELSE NULL
    END AS 資格2
FROM 
    SHAIN_SHUTOKU_SHIKAKU AS T1
LEFT OUTER JOIN SHAIN_SHUTOKU_SHIKAKU AS T2 
    ON T1.社員コード = T2.社員コード 
        AND T2.資格 = 'AP'
WHERE
    T1.資格 = 'FE';

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


社員コード,資格1,資格2
S001,FE,AP
S002,FE,None
S003,FE,None
